In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
import netron

 
wave=8192
wavetoChn=int(wave/8)
print(wavetoChn)

def cal_len(x):
    return x//2 -1

class Model(nn.Module):
    def __init__(self):
        """
            CNN模型构造
        """
        super(Model, self).__init__()
        self.conv_layer1 = nn.Sequential(
            # input shape(32, 1, 256) -> [batch_size, channel, features] 8192
            # 参考->https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, padding=1),   # 卷积后(32, 16, 256) 8192
            nn.BatchNorm1d(16),
            nn.ReLU()
        )
        # 下采样down-sampling
        self.sampling_layer1 = nn.Sequential(
            # input shape(32, 16, 256) 8192
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # size随便选的, 这里output应该是(32, 32, 128) 4096
        )

        self.conv_layer2 = nn.Sequential(
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),   # 输出(32, 64, 128) 4096
            nn.BatchNorm1d(64),
            nn.ReLU()
        )

        self.sampling_layer2 = nn.Sequential(
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1),  # 输出(32, 128, 128)
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # 输出(32, 64, 64)
        )

        self.conv_layer3 = nn.Sequential(
            nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1),  # 输出(32, 256, 64)
            nn.BatchNorm1d(256),
            nn.ReLU()
        )

        self.sampling_layer3 = nn.Sequential(
            nn.Conv1d(in_channels=256, out_channels=512, kernel_size=3, padding=1),  # 输出(32, 512, 64)
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # 输出(32, 512, 32)
        )
        # 全连接层
        self.full_layer = nn.Sequential(
            nn.Linear(in_features=512*wavetoChn, out_features=256*wavetoChn),
            nn.ReLU(),
            nn.Linear(in_features=256*wavetoChn, out_features=128*wavetoChn),
            nn.ReLU(),
            nn.Linear(in_features=128*wavetoChn, out_features=64*wavetoChn),
            nn.ReLU(),
            nn.Linear(in_features=64*wavetoChn, out_features=6)
        )
        # 这个是输出label预测概率, 不知道这写法对不对
        self.pred_layer = nn.Softmax(dim=1)

    def forward(self, x):
        """
            前向传播
        :param x: batch
        :return: training == Ture 返回的是全连接层输出， training == False 加上一个Softmax(), 返回各个label概率.
        """
        x = x.unsqueeze(dim=1)  # 升维. input shape(32, 205), output shape(32, 1, 205)
        x = self.conv_layer1(x)
        x = self.sampling_layer1(x)
        x = self.conv_layer2(x)
        x = self.sampling_layer2(x)
        x = self.conv_layer3(x)
        x = self.sampling_layer3(x)
        x = x.view(x.size(0), -1)   # output(32, 12800)
        x = self.full_layer(x)

        if self.training:
            return x    # CrossEntropyLoss自带LogSoftmax, 训练的时候不用输出概率(我也不知道这个写法对不对, 我是试错出来的.)
        else:
            return self.pred_layer(x)

 
class Net(nn.Module):
    def __init__(self, ecgsamples=5000):
        super(Net, self).__init__()
        self.ecgsamples = ecgsamples
        self.conv1 = nn.Conv1d(12, 32, kernel_size=3, stride=1, padding=0)
#         self.conv1_bn = nn.batchnorm()
        self.conv2 = nn.Conv1d(32, 32, kernel_size=3, stride=1, padding=0)
        self.conv3 = nn.Conv1d(32, 32, kernel_size=3, stride=1, padding=0)
        self.fc1 = nn.Linear(cal_len(cal_len(cal_len(self.ecgsamples)))-3, 5)
        
    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), 2, stride=2, padding=0)
        x = F.max_pool1d(F.relu(self.conv2(x)), 2, stride=2, padding=0)
        x = F.max_pool1d(F.relu(self.conv2(x)), 2, stride=2, padding=0)
        x = F.max_pool1d(F.relu(self.conv2(x)), 2, stride=1, padding=0)
        x = x.view(-1, x.shape[-1]*x.shape[-2])
        return x


class Bi_Lstm(nn.Module):
    def __init__(self):
        super(Bi_Lstm, self).__init__()
        self.lstm = nn.LSTM(input_size=8192, hidden_size=2048,
                            num_layers=3)  # 加了双向，输出的节点数翻2倍
        self.l1 = nn.Linear(2048, 1024)  # 特征输入
        self.l2 = nn.ReLU()  # 激活函数
        self.l3 = nn.BatchNorm1d(1024)  # 批标准化
        self.l4 = nn.Linear(1024, 512)
        self.l5 = nn.ReLU()
        self.l6 = nn.BatchNorm1d(512)
        self.l7 = nn.Linear(512, 6)  # 输出6个节点
        self.l8 = nn.BatchNorm1d(6)

    def forward(self, x):
        out, _ = self.lstm(x)
        # 选择最后一个时间点的output
        out = self.l1(out[:, -1, :])
        out = self.l2(out)
        out = self.l3(out)
        out = self.l4(out)
        out = self.l5(out)
        out = self.l6(out)
        out = self.l7(out)
        out = self.l8(out)
        return out
        
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
device = "cuda:0"
# model = Net(ecgsamples=10000).to(device)
# summary(model, input_size=(12, 10000))
# model1 =Model()
# model1 = model1.to(device)
# summary(model1, input_size=(1, 8192))
model2 = Bi_Lstm().to(device)
# model2 = model2.to(device)
data = torch.rand(1,64, 8192)
data=data.to(device)

summary(model2, data)

onnx_path = "onnx_model_name.onnx" # 文件名
torch.onnx.export(model2, data, onnx_path) # 导出神经网络模型为onnx格式
netron.start(onnx_path) # 启动netron

1024
Layer (type:depth-idx)                   Output Shape              Param #
├─LSTM: 1-1                              [-1, 64, 2048]            151,044,096
├─Linear: 1-2                            [-1, 1024]                2,098,176
├─ReLU: 1-3                              [-1, 1024]                --
├─BatchNorm1d: 1-4                       [-1, 1024]                2,048
├─Linear: 1-5                            [-1, 512]                 524,800
├─ReLU: 1-6                              [-1, 512]                 --
├─BatchNorm1d: 1-7                       [-1, 512]                 1,024
├─Linear: 1-8                            [-1, 6]                   3,078
├─BatchNorm1d: 1-9                       [-1, 6]                   12
Total params: 153,673,234
Trainable params: 153,673,234
Non-trainable params: 0
Total mult-adds (M): 153.62
Input size (MB): 2.00
Forward/backward pass size (MB): 1.02
Params size (MB): 586.22
Estimated Total Size (MB): 589.24


/home/tzr/.local/lib/python3.8/site-packages/torch/onnx/symbolic_opset9.py:4315: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
/home/tzr/.local/lib/python3.8/site-packages/torch/onnx/_internal/jit_utils.py:258: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (Triggered internally at ../torch/csrc/jit/passes/onnx/shape_type_inference.cpp:1884.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
/home/tzr/.local/lib/python3.8/site-packages/torch/onnx/utils.py:687: UserWarning: The shape inference of prim::Constant type is missing, so it may result in wrong shape infer

Serving 'onnx_model_name.onnx' at http://localhost:8080


('localhost', 8080)

: 